In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #(xxxx is your specific GPU ID)

In [2]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras.layers import Activation, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from MyEarlyStopping import MyEarlyStopping
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# from keras.optimizers import adam

import timeit
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder  

# load dataset 

In [3]:
train_number = 5
train_image = 20 #10:20%, 20: 40%, 30:60%
train_image_s2 = 30 #10:20%, 20: 40%, 30:60%

## Training dataset (40%)

In [4]:
train_data = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_data.flow_from_directory('/data/home/cou/yuli5/Audio/Comparision/Different_Inputs/Audio_MFCC/train',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 1714 images belonging to 10 classes.


In [5]:
image_names = train_generator.filenames
image_no = [i.split("/")[1].split("_")[2].split(".")[0] for i in image_names]
image_no = np.array(list(map(int, image_no)))
ALL_participant_class = [i.split("/")[1].split("_")[1] for i in image_names]
ALL_participant_class = np.array(list(map(int, ALL_participant_class)))
command_class = train_generator.classes
All_participant_class = tf.keras.utils.to_categorical(ALL_participant_class-1, num_classes=train_number)
All_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
All_command_uniform = All_command_class*0+1/10

In [6]:
All_Inputs = [next(train_generator)[0][0] for _ in range(len(train_generator))]
All_Inputs = np.array(All_Inputs)

In [7]:
select_indexs_train = ((image_no<train_image)&(ALL_participant_class!=2))|((image_no<train_image_s2)&(ALL_participant_class==2))
Train_Inputs = All_Inputs[select_indexs_train]
Train_participant_class = All_participant_class[select_indexs_train]
Train_participant_uniform = Train_participant_class*0+1/train_number
Train_command_class = All_command_class[select_indexs_train]
Train_command_uniform = Train_command_class*0+1/10
# sum(Train_participant_class)


## Validation dataset (20%)

In [8]:
val_data = ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator = val_data.flow_from_directory('/data/home/cou/yuli5/Audio/Comparision/Different_Inputs/Audio_MFCC/validation',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 543 images belonging to 10 classes.


In [9]:
image_names = val_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
participant_class = np.array(list(map(int, participant_class)))
command_class = val_generator.classes
Val_participant_class = tf.keras.utils.to_categorical(participant_class-1, num_classes=train_number)
Val_participant_uniform = Val_participant_class*0+1/train_number
Val_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
Val_command_uniform = Val_command_class*0+1/10

In [10]:
Val_Inputs = [next(val_generator)[0][0] for _ in range(len(val_generator))]
Val_Inputs = np.array(Val_Inputs)

## Test dataset (20%)

In [11]:
test_data = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data.flow_from_directory('/data/home/cou/yuli5/Audio/Comparision/Different_Inputs/Audio_MFCC/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 543 images belonging to 10 classes.


In [12]:
image_names = test_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
test_unit_participant_class = np.array(list(map(int, participant_class)))
test_unit_command_class = test_generator.classes
Test_participant_class = tf.keras.utils.to_categorical(test_unit_participant_class-1, num_classes=train_number)
Test_participant_uniform = Test_participant_class*0+1/train_number
Test_command_class = tf.keras.utils.to_categorical(test_unit_command_class, num_classes=10)
Test_command_uniform = Test_command_class*0+1/10

In [13]:
Test_Inputs = [next(test_generator)[0][0] for _ in range(len(test_generator))]
Test_Inputs = np.array(Test_Inputs)

In [14]:
Test_Inputs_1 = Test_Inputs[np.where(test_unit_participant_class == 1)]
Test_command_class_1 = Test_command_class[np.where(test_unit_participant_class == 1)]
Test_Inputs_2 = Test_Inputs[np.where(test_unit_participant_class == 2)]
Test_command_class_2 = Test_command_class[np.where(test_unit_participant_class == 2)]
Test_Inputs_3 = Test_Inputs[np.where(test_unit_participant_class == 3)]
Test_command_class_3 = Test_command_class[np.where(test_unit_participant_class == 3)]
Test_Inputs_4 = Test_Inputs[np.where(test_unit_participant_class == 4)]
Test_command_class_4 = Test_command_class[np.where(test_unit_participant_class == 4)]
Test_Inputs_5 = Test_Inputs[np.where(test_unit_participant_class == 5)]
Test_command_class_5 = Test_command_class[np.where(test_unit_participant_class == 5)]

# pd file store performance

In [15]:
# pd file store performance
# Perfomance = pd.DataFrame(columns = ['Feature_Extract','Time','Partcp_Acc_p15','Command_Acc_p15','Partcp_Acc_p1','Command_Acc_p1',
#                                       'Partcp_Acc_p2','Command_Acc_p2','Partcp_Acc_p3','Command_Acc_p3',
#                                       'Partcp_Acc_p4','Command_Acc_p4','Partcp_Acc_p5','Command_Acc_p5'])

Perfomance = pd.read_csv('Performance_0907_Feature_extract.csv')
# Perfomance
                                                   

# Create model 

In [16]:
actv_fun_1_1 = "relu" 
actv_fun_1_2 = "sigmoid"
shape_1_1 = 128
shape_1_2 = 256
actv_fun_2_1 = "sigmoid" 
actv_fun_2_2 = "sigmoid"
shape_2_1 = 512
shape_2_2 = 512


## Group model 

In [17]:
high_acc = 0
for run in range(0,10):
    # feature extraction layers
    INCEPTION_model = InceptionResNetV2(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in INCEPTION_model.layers:
        layer.trainable = False

    d = INCEPTION_model.output.shape[-1] # dimension of last layer

    ###################### model 1 ###################### 
    layer_1_0 = tf.keras.layers.Dense(d,name="weight_1")(INCEPTION_model.output) #times weight before flatten
    layer_1_1 = tf.keras.layers.Flatten(name='flatten_1')(layer_1_0)

    Dense_1_1 = tf.keras.layers.Dense(shape_1_1, activation=actv_fun_1_1,name='fc1_1')
    layer_1_2 = Dense_1_1(layer_1_1)
    Dense_1_2 = tf.keras.layers.Dense(shape_1_2, activation=actv_fun_1_2,name='fc1_2')
    layer_1_3 = Dense_1_2(layer_1_2)

    Dense_1_3 = tf.keras.layers.Dense(train_number, activation='softmax' ,name='participant_output')
    out_layer_1 = Dense_1_3(layer_1_3)

    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(INCEPTION_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)

    ###################### model 1' ###################### 
    layer_1_2_  = Dense_2_1(layer_1_1)
    layer_1_3_  = Dense_2_2(layer_1_2_)
    out_layer_1_ = Dense_2_3(layer_1_3_)

    ###################### model 1' ###################### 
    layer_2_2_  = Dense_1_1(layer_2_1)
    layer_2_3_  = Dense_1_2(layer_2_2_)
    out_layer_2_ = Dense_1_3(layer_2_3_)

    INCEPTION_model = tf.keras.Model(INCEPTION_model.input, [out_layer_1,out_layer_2,out_layer_1_,out_layer_2_])


    # INCEPTION_model.summary() 

    w_1, w_2, w_1_, w_2_ = 1,1,1,1
    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [MyEarlyStopping(monitor1 = 'val_' + INCEPTION_model.layers[-1].name+'_accuracy',
                                  monitor2 = 'val_' + INCEPTION_model.layers[-2].name+'_accuracy',
                                  patience=5,restore_best_weights=True)]
    INCEPTION_model.compile(optimizer=opt, loss=["categorical_crossentropy","categorical_crossentropy","mse","mse"],
                         loss_weights=[w_1, w_2, w_1_, w_2_], metrics=['accuracy'])


    start = timeit.default_timer()

    history1 = INCEPTION_model.fit(Train_Inputs, 
                           {INCEPTION_model.layers[-2].name:Train_participant_class, 
                            INCEPTION_model.layers[-1].name:Train_command_class,
                            INCEPTION_model.layers[-1].name+"_1":Train_command_uniform, 
                            INCEPTION_model.layers[-2].name+"_1":Train_participant_uniform}, 
                            validation_data=(Val_Inputs,
                                             {INCEPTION_model.layers[-2].name:Val_participant_class,
                                              INCEPTION_model.layers[-1].name:Val_command_class,
                                              INCEPTION_model.layers[-1].name+"_1":Val_command_uniform,
                                              INCEPTION_model.layers[-2].name+"_1":Val_participant_uniform}), 
                           callbacks=callbacks,
                           batch_size=64,
                           epochs=10)

    for layer in INCEPTION_model.layers[0:19]:
        layer.trainable = True

    history2 = INCEPTION_model.fit(Train_Inputs, 
                               {INCEPTION_model.layers[-2].name:Train_participant_class, 
                                INCEPTION_model.layers[-1].name:Train_command_class,
                                INCEPTION_model.layers[-1].name+"_1":Train_command_uniform, 
                                INCEPTION_model.layers[-2].name+"_1":Train_participant_uniform}, 
                                validation_data=(Val_Inputs,
                                                 {INCEPTION_model.layers[-2].name:Val_participant_class,
                                                  INCEPTION_model.layers[-1].name:Val_command_class,
                                                  INCEPTION_model.layers[-1].name+"_1":Val_command_uniform,
                                                  INCEPTION_model.layers[-2].name+"_1":Val_participant_uniform}), 
                               callbacks=callbacks,
                               batch_size=64,
                               epochs=10)

    stop = timeit.default_timer()
    run_time = stop - start

    ##################### test performance ############################
    predictions = INCEPTION_model.predict(Test_Inputs)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = test_unit_participant_class
    acc_p15_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)

    predictions = INCEPTION_model.predict(Test_Inputs)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class
    acc_p15_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)


    # test on p1
    ## speaker classification
    predictions = INCEPTION_model.predict(Test_Inputs_1)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([1]*len(predicted_classes))
    acc_p1_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = INCEPTION_model.predict(Test_Inputs_1)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_1, axis=-1)
    acc_p1_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p2
    ## speaker classification
    predictions = INCEPTION_model.predict(Test_Inputs_2)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([2]*len(predicted_classes))
    acc_p2_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = INCEPTION_model.predict(Test_Inputs_2)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_2, axis=-1)
    acc_p2_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p3
    ## speaker classification
    predictions = INCEPTION_model.predict(Test_Inputs_3)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([3]*len(predicted_classes))
    acc_p3_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = INCEPTION_model.predict(Test_Inputs_3)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_3, axis=-1)
    acc_p3_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p4
    ## speaker classification
    predictions = INCEPTION_model.predict(Test_Inputs_4)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([4]*len(predicted_classes))
    acc_p4_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = INCEPTION_model.predict(Test_Inputs_4)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_4, axis=-1)
    acc_p4_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p5
    ## speaker classification
    predictions = INCEPTION_model.predict(Test_Inputs_5)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([5]*len(predicted_classes))
    acc_p5_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = INCEPTION_model.predict(Test_Inputs_5)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_5, axis=-1)
    acc_p5_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   


    Perfomance = Perfomance.append({'Feature_Extract': "INCEPTION_Resnet_v2",'Size':'20&s2_30','Time':run_time,
                                    'Partcp_Acc_p15':acc_p15_s,'Command_Acc_p15':acc_p15_c,'Partcp_Acc_p1':acc_p1_s,
                                    'Command_Acc_p1':acc_p1_c,'Partcp_Acc_p2':acc_p2_s,'Command_Acc_p2':acc_p2_c,
                                    'Partcp_Acc_p3':acc_p3_s,'Command_Acc_p3':acc_p3_c,'Partcp_Acc_p4':acc_p4_s,
                                    'Command_Acc_p4':acc_p4_c,'Partcp_Acc_p5':acc_p5_s,'Command_Acc_p5':acc_p5_c}, ignore_index=True)


    if high_acc < acc_p15_c + acc_p15_s:
        INCEPTION_model.save('Initial_group_model_INCEPTION_0907.h5')
        high_acc = acc_p15_c + acc_p15_s
        best_index = run
        pd.DataFrame.from_dict(history1.history).to_csv('mix_20&s2_30_history1_Initial_group_model_INCEPTION_0907.csv',index=False)
        pd.DataFrame.from_dict(history2.history).to_csv('mix_20&s2_30_history2_Initial_group_model_INCEPTION_0907.csv',index=False)
        
    del INCEPTION_model
    run = run + 1

219062272/219055592 [==============================] - 5s 0us/step
Epoch 1/10
18/18 [==============================] - 8s 421ms/step - loss: 3.4976 - participant_output_loss: 1.4571 - command_output_loss: 2.0358 - command_output_1_loss: 8.0854e-04 - participant_output_1_loss: 0.0038 - participant_output_accuracy: 0.3991 - command_output_accuracy: 0.3364 - command_output_1_accuracy: 0.0064 - participant_output_1_accuracy: 0.1218 - val_loss: 3.4499 - val_participant_output_loss: 1.3770 - val_command_output_loss: 2.0722 - val_command_output_1_loss: 2.5812e-04 - val_participant_output_1_loss: 3.8239e-04 - val_participant_output_accuracy: 0.5083 - val_command_output_accuracy: 0.3831 - val_command_output_1_accuracy: 0.6262 - val_participant_output_1_accuracy: 0.1436
Epoch 2/10
18/18 [==============================] - 4s 245ms/step - loss: 2.5157 - participant_output_loss: 1.1116 - command_output_loss: 1.4035 - command_output_1_loss: 2.3988e-04 - participant_output_1_loss: 3.4867e-04 - partic

18/18 [==============================] - 4s 196ms/step - loss: 0.2479 - participant_output_loss: 0.1358 - command_output_loss: 0.1115 - command_output_1_loss: 3.3971e-04 - participant_output_1_loss: 1.5295e-04 - participant_output_accuracy: 0.9918 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.1009 - participant_output_1_accuracy: 0.1936 - val_loss: 2.9804 - val_participant_output_loss: 0.8320 - val_command_output_loss: 2.1478 - val_command_output_1_loss: 3.5513e-04 - val_participant_output_1_loss: 2.0537e-04 - val_participant_output_accuracy: 0.8029 - val_command_output_accuracy: 0.5341 - val_command_output_1_accuracy: 0.1068 - val_participant_output_1_accuracy: 0.2247
Epoch 3/10
18/18 [==============================] - 4s 210ms/step - loss: 0.2043 - participant_output_loss: 0.1127 - command_output_loss: 0.0911 - command_output_1_loss: 3.4951e-04 - participant_output_1_loss: 1.4379e-04 - participant_output_accuracy: 0.9918 - command_output_accuracy: 1.0000 - command_

Epoch 4/10
18/18 [==============================] - 4s 249ms/step - loss: 1.5569 - participant_output_loss: 0.7096 - command_output_loss: 0.8466 - command_output_1_loss: 5.9028e-04 - participant_output_1_loss: 1.6373e-04 - participant_output_accuracy: 0.8382 - command_output_accuracy: 0.8827 - command_output_1_accuracy: 0.0391 - participant_output_1_accuracy: 0.2127 - val_loss: 2.9177 - val_participant_output_loss: 1.0414 - val_command_output_loss: 1.8755 - val_command_output_1_loss: 6.4822e-04 - val_participant_output_1_loss: 2.1382e-04 - val_participant_output_accuracy: 0.6869 - val_command_output_accuracy: 0.4825 - val_command_output_1_accuracy: 0.3978 - val_participant_output_1_accuracy: 0.0902
Epoch 5/10
18/18 [==============================] - 4s 250ms/step - loss: 1.2027 - participant_output_loss: 0.5852 - command_output_loss: 0.6167 - command_output_1_loss: 6.3662e-04 - participant_output_1_loss: 1.5582e-04 - participant_output_accuracy: 0.8764 - command_output_accuracy: 0.9255

Epoch 6/10
18/18 [==============================] - 4s 216ms/step - loss: 0.1069 - participant_output_loss: 0.0520 - command_output_loss: 0.0545 - command_output_1_loss: 3.2710e-04 - participant_output_1_loss: 5.3989e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0945 - participant_output_1_accuracy: 0.2164 - val_loss: 3.0685 - val_participant_output_loss: 0.7949 - val_command_output_loss: 2.2731 - val_command_output_1_loss: 3.5001e-04 - val_participant_output_1_loss: 1.1539e-04 - val_participant_output_accuracy: 0.8269 - val_command_output_accuracy: 0.5414 - val_command_output_1_accuracy: 0.0994 - val_participant_output_1_accuracy: 0.1436
Epoch 7/10
18/18 [==============================] - 4s 247ms/step - loss: 0.0921 - participant_output_loss: 0.0442 - command_output_loss: 0.0475 - command_output_1_loss: 3.1839e-04 - participant_output_1_loss: 4.7042e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000

Epoch 8/10
18/18 [==============================] - 4s 240ms/step - loss: 0.5607 - participant_output_loss: 0.3055 - command_output_loss: 0.2548 - command_output_1_loss: 2.3617e-04 - participant_output_1_loss: 1.7513e-04 - participant_output_accuracy: 0.9591 - command_output_accuracy: 0.9891 - command_output_1_accuracy: 0.1018 - participant_output_1_accuracy: 0.2036 - val_loss: 2.7790 - val_participant_output_loss: 0.8012 - val_command_output_loss: 1.9772 - val_command_output_1_loss: 2.6976e-04 - val_participant_output_1_loss: 2.1666e-04 - val_participant_output_accuracy: 0.7993 - val_command_output_accuracy: 0.5322 - val_command_output_1_accuracy: 0.1031 - val_participant_output_1_accuracy: 0.2302
Epoch 9/10
18/18 [==============================] - 4s 214ms/step - loss: 0.4354 - participant_output_loss: 0.2326 - command_output_loss: 0.2024 - command_output_1_loss: 2.5965e-04 - participant_output_1_loss: 1.5937e-04 - participant_output_accuracy: 0.9782 - command_output_accuracy: 0.9955

...

Epoch 9/10
18/18 [==============================] - 4s 216ms/step - loss: 0.4408 - participant_output_loss: 0.2441 - command_output_loss: 0.1961 - command_output_1_loss: 3.0568e-04 - participant_output_1_loss: 1.7403e-04 - participant_output_accuracy: 0.9618 - command_output_accuracy: 0.9945 - command_output_1_accuracy: 0.1273 - participant_output_1_accuracy: 0.2373 - val_loss: 2.8583 - val_participant_output_loss: 0.8558 - val_command_output_loss: 2.0020 - val_command_output_1_loss: 3.1734e-04 - val_participant_output_1_loss: 1.9456e-04 - val_participant_output_accuracy: 0.7827 - val_command_output_accuracy: 0.5359 - val_command_output_1_accuracy: 0.1418 - val_participant_output_1_accuracy: 0.2210
Epoch 10/10
18/18 [==============================] - 3s 180ms/step - loss: 0.3526 - participant_output_loss: 0.1981 - command_output_loss: 0.1540 - command_output_1_loss: 3.2791e-04 - participant_output_1_loss: 1.5104e-04 - participant_output_accuracy: 0.9782 - command_output_accuracy: 0.999

Epoch 1/10
18/18 [==============================] - 7s 381ms/step - loss: 3.6828 - participant_output_loss: 1.5658 - command_output_loss: 2.1042 - command_output_1_loss: 0.0012 - participant_output_1_loss: 0.0116 - participant_output_accuracy: 0.3455 - command_output_accuracy: 0.3400 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.1682 - val_loss: 3.5505 - val_participant_output_loss: 1.4473 - val_command_output_loss: 2.0970 - val_command_output_1_loss: 4.0162e-04 - val_participant_output_1_loss: 0.0058 - val_participant_output_accuracy: 0.4309 - val_command_output_accuracy: 0.3499 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.8306
Epoch 2/10
18/18 [==============================] - 4s 206ms/step - loss: 2.7343 - participant_output_loss: 1.2305 - command_output_loss: 1.4999 - command_output_1_loss: 2.3903e-04 - participant_output_1_loss: 0.0037 - participant_output_accuracy: 0.5491 - command_output_accuracy: 0.6918 - comma

Epoch 3/10
18/18 [==============================] - 4s 211ms/step - loss: 0.2623 - participant_output_loss: 0.1658 - command_output_loss: 0.0959 - command_output_1_loss: 3.3850e-04 - participant_output_1_loss: 2.3991e-04 - participant_output_accuracy: 0.9900 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.1064 - participant_output_1_accuracy: 0.2209 - val_loss: 2.8982 - val_participant_output_loss: 0.7409 - val_command_output_loss: 2.1567 - val_command_output_1_loss: 3.5428e-04 - val_participant_output_1_loss: 3.1705e-04 - val_participant_output_accuracy: 0.8214 - val_command_output_accuracy: 0.5359 - val_command_output_1_accuracy: 0.1050 - val_participant_output_1_accuracy: 0.1547
Epoch 4/10
18/18 [==============================] - 3s 183ms/step - loss: 0.2065 - participant_output_loss: 0.1311 - command_output_loss: 0.0748 - command_output_1_loss: 3.3810e-04 - participant_output_1_loss: 2.1926e-04 - participant_output_accuracy: 0.9945 - command_output_accuracy: 1.0000

Epoch 5/10
18/18 [==============================] - 4s 225ms/step - loss: 1.1268 - participant_output_loss: 0.5598 - command_output_loss: 0.5665 - command_output_1_loss: 2.8673e-04 - participant_output_1_loss: 1.7600e-04 - participant_output_accuracy: 0.8864 - command_output_accuracy: 0.9355 - command_output_1_accuracy: 0.1518 - participant_output_1_accuracy: 0.1991 - val_loss: 2.7935 - val_participant_output_loss: 0.9370 - val_command_output_loss: 1.8562 - val_command_output_1_loss: 2.1554e-04 - val_participant_output_1_loss: 1.9763e-04 - val_participant_output_accuracy: 0.7459 - val_command_output_accuracy: 0.5175 - val_command_output_1_accuracy: 0.1455 - val_participant_output_1_accuracy: 0.1786
Epoch 6/10
18/18 [==============================] - 4s 231ms/step - loss: 0.8708 - participant_output_loss: 0.4413 - command_output_loss: 0.4290 - command_output_1_loss: 2.1635e-04 - participant_output_1_loss: 1.7408e-04 - participant_output_accuracy: 0.9273 - command_output_accuracy: 0.9518

Epoch 7/10
18/18 [==============================] - 4s 217ms/step - loss: 0.0927 - participant_output_loss: 0.0468 - command_output_loss: 0.0456 - command_output_1_loss: 2.9115e-04 - participant_output_1_loss: 5.0223e-05 - participant_output_accuracy: 0.9991 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0982 - participant_output_1_accuracy: 0.1927 - val_loss: 3.0699 - val_participant_output_loss: 0.7722 - val_command_output_loss: 2.2973 - val_command_output_1_loss: 3.1881e-04 - val_participant_output_1_loss: 8.4378e-05 - val_participant_output_accuracy: 0.8250 - val_command_output_accuracy: 0.5396 - val_command_output_1_accuracy: 0.0773 - val_participant_output_1_accuracy: 0.1934
Epoch 8/10
18/18 [==============================] - 4s 219ms/step - loss: 0.0813 - participant_output_loss: 0.0410 - command_output_loss: 0.0400 - command_output_1_loss: 2.7954e-04 - participant_output_1_loss: 4.4398e-05 - participant_output_accuracy: 0.9991 - command_output_accuracy: 1.0000

In [18]:
INCEPTION_model = tf.keras.models.load_model('Initial_group_model_INCEPTION_0907.h5')
predictions = INCEPTION_model.predict(Test_Inputs)[0]
predicted_classes = np.argmax(predictions, axis=1)+1
acc_s = round(sum(x == y for x, y in zip(test_unit_participant_class, predicted_classes)) / len(test_unit_participant_class),4)

predictions = INCEPTION_model.predict(Test_Inputs)[1]
predicted_classes = np.argmax(predictions, axis=1)
acc_c = round(sum(x == y for x, y in zip(test_unit_command_class, predicted_classes)) / len(test_unit_command_class),4)
overall_acc = acc_c + acc_s
print(acc_s,acc_c)

0.9042 0.9503


In [19]:
Perfomance

,Feature_Extract,Time,Partcp_Acc_p15,Command_Acc_p15,Partcp_Acc_p1,Command_Acc_p1,Partcp_Acc_p2,Command_Acc_p2,Partcp_Acc_p3,Command_Acc_p3,Partcp_Acc_p4,Command_Acc_p4,Partcp_Acc_p5,Command_Acc_p5,Size
0,VGG16,63.237659,0.8895,0.9134,0.9189,0.9279,0.9722,0.7685,0.9217,0.9130,0.7615,0.9725,0.87,0.99,20&s2_30
1,VGG16,62.212071,0.8803,0.9116,0.8649,0.9189,0.9722,0.7685,0.9304,0.9304,0.7339,0.9633,0.90,0.98,20&s2_30
2,VGG16,61.656819,0.8840,0.9061,0.9009,0.9279,0.9444,0.7593,0.9304,0.9217,0.7798,0.9541,0.86,0.97,20&s2_30
3,VGG16,61.511568,0.8803,0.9263,0.8649,0.9369,0.9537,0.7963,0.8870,0.9478,0.7706,0.9725,0.93,0.98,20&s2_30
4,VGG16,60.123125,0.8785,0.9171,0.7928,0.9009,0.9537,0.7870,0.9043,0.9478,0.8440,0.9817,0.90,0.97,20&s2_30
5,VGG16,59.735848,0.8840,0.9190,0.9099,0.9279,0.9444,0.7870,0.9217,0.9304,0.7431,0.9725,0.90,0.98,20&s2_30
6,VGG16,57.168746,0.8803,0.9079,0.8468,0.9189,0.9444,0.7685,0.9304,0.9130,0.7798,0.9725,0.90,0.97,20&s2_30
7,VGG16,57.110326,0.8748,0.9042,0.8288,0.8919,0.9074,0.7593,0.8957,0.9391,0.8440,0.9725,0.90,0.96,20&s2_30
8,VGG16,57.662082,0.8766,0.9098,0.9009,0.9279,0.9537,0.7593,0.8957,0.9478,0.7706,0.9450,0.86,0.97,20&s2_30
9,VGG16,58.186677,0.8895,0.9061,0.8559,0.9009,0.9537,0.7685,0.9478,0.9130,0.7890,0.9725,0.90,0.98,20&s2_30


In [20]:
Perfomance.to_csv('Performance_0907_Feature_extract.csv',index=False)

In [21]:
best_index

3